In [3]:
import pandas as pd
import string
import numpy as np
import simpletransformers

In [1]:
pip install simpletransformers

     |████████████████████████████████| 225kB 11.1MB/s 
     |████████████████████████████████| 71kB 6.5MB/s 
     |████████████████████████████████| 1.3MB 19.6MB/s 
     |████████████████████████████████| 317kB 17.5MB/s 
     |████████████████████████████████| 2.9MB 41.7MB/s 
     |████████████████████████████████| 51kB 7.1MB/s 
     |████████████████████████████████| 7.4MB 40.2MB/s 
     |████████████████████████████████| 1.8MB 25.0MB/s 
     |████████████████████████████████| 1.1MB 33.0MB/s 
     |████████████████████████████████| 890kB 37.2MB/s 
     |████████████████████████████████| 6.9MB 39.9MB/s 
     |████████████████████████████████| 133kB 45.4MB/s 
     |████████████████████████████████| 112kB 44.8MB/s 
     |████████████████████████████████| 102kB 9.9MB/s 
     |████████████████████████████████| 4.5MB 36.6MB/s 
     |████████████████████████████████| 163kB 44.5MB/s 
     |████████████████████████████████| 133kB 40.7MB/s 
     |████████████████████████████████| 102kB 10.8MB/

In [35]:
!git clone https://github.com/Navjot862/Courseera_analytics_tools.git

fatal: destination path 'Courseera_analytics_tools' already exists and is not an empty directory.


In [36]:
Review=pd.read_csv("/content/Courseera_analytics_tools/Train_Data.csv")
Review=Review[['Review']]

In [37]:
##Convert everything to lower cases 
Review['Review']=Review['Review'].str.lower()

In [38]:
Proffessor=['professors','instructor',"ittner","bushee","lecturer","teacher"]

Courses=['autocad','3d printing','traditional tooling','economy','business','course','analytics','accounting','accounting analytics',
         'business analytics', 'business market analysis','advanced neurobiology']

Content=['lecture','videos','content','material','assignments','information']

In [39]:
Review_list=list(Review['Review'])

In [40]:
##Creating an empty data frame
Review_Data_Train=pd.DataFrame(columns=['Sentence_Id','Word','Label'])

In [41]:
for i in range(len(Review_list)):
    Chk=pd.DataFrame(columns=['Sentence_Id','Word','Label'])
#     print(i)
    l2=Review_list[i].split()
    for j in range(len(l2)):
        Chk.at[j, 'Sentence_Id'] = i
        Chk.at[j, 'Word'] = l2[j]
#     print(Review_Data_Train.head(2))
    Review_Data_Train=pd.concat([Review_Data_Train,Chk])
    

In [42]:
Review_Data_Train['Label']='O'

Data=pd.DataFrame(columns=['Sentence_Id','Word','Label'])

for m in np.unique(Review_Data_Train['Sentence_Id']):
#     print(m)
    first_course=0
    first_prof=0
    first_content=0
    Review_Data_Sentence=Review_Data_Train[Review_Data_Train['Sentence_Id']==m]

    for i in range(Review_Data_Sentence.shape[0]):
        
        ##Course
        
        if Review_Data_Sentence['Word'].iloc[i] in Courses and first_course==0:
            Review_Data_Sentence['Label'].iloc[i]='B-Course'
            first_course=first_course+1
            
        elif Review_Data_Sentence['Word'].iloc[i] in Courses and first_course>=0:
            Review_Data_Sentence['Label'].iloc[i]='I-Course'
            first_course=first_course+1   
            
        ##Professor    
        
        if Review_Data_Sentence['Word'].iloc[i] in Proffessor and first_prof==0:
            Review_Data_Sentence['Label'].iloc[i]='B-Prof'
            first_prof=first_prof+1
            
        elif Review_Data_Sentence['Word'].iloc[i] in Proffessor and first_prof>=0:
            Review_Data_Sentence['Label'].iloc[i]='I-Prof'
            first_prof=first_prof+1
        
        ##Content
        
        if Review_Data_Sentence['Word'].iloc[i] in Content and first_content==0:
            Review_Data_Sentence['Label'].iloc[i]='B-Content'
            first_content=first_content+1   
            
            
        elif Review_Data_Sentence['Word'].iloc[i] in Content and first_content>=0:
            Review_Data_Sentence['Label'].iloc[i]='I-Content'
            first_content=first_content+1  
            
    Data=pd.concat([Data,Review_Data_Sentence])        
    

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to b

In [29]:
Train_l1=[]
for x in range(20):
    Train_l1.append(x)
print(Train_l1)

Test_l1=[]
for x in range(20,25):
    Test_l1.append(x)
print(Test_l1)

eval_l1=[]
for x in range(25,30):
    eval_l1.append(x)
print(eval_l1)

Train_df=Data[Data['Sentence_Id'].isin(Train_l1)]
Test_df=Data[Data['Sentence_Id'].isin(Test_l1)]
eval_df=Data[Data['Sentence_Id'].isin(eval_l1)]

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
[20, 21, 22, 23, 24]
[25, 26, 27, 28, 29]


In [43]:
Train_df=Data

In [44]:
Train_df.columns=["sentence_id", "words", "labels"]
Test_df.columns=["sentence_id", "words", "labels"]
eval_df.columns=["sentence_id", "words", "labels"]

In [45]:
np.unique(Train_df['labels'])

array(['B-Content', 'B-Course', 'B-Prof', 'I-Content', 'I-Course',
       'I-Prof', 'O'], dtype=object)

In [46]:
from simpletransformers.ner import NERModel, NERArgs

# Configure the model
model_args = NERArgs(overwrite_output_dir=True,num_train_epochs=1)
# model_args.train_batch_size = 16
# model_args.evaluate_during_training = True
model_args.labels_list=list(np.unique(Train_df['labels']))

model = NERModel("roberta", "roberta-base", args=model_args,use_cuda=False)

# Train the model
model.train_model(Train_df)

# # Evaluate the model
# result, model_outputs, preds_list = model.eval_model(eval_df)


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForTokenClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:216: UserWarning: Please also save or load the state of the optimizer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


(219, 0.12773253911091909)

In [49]:
predictions, raw_outputs = model.predict(["The course is fine as far as content. My complaint is that there is no one to contact regarding papers or questions. You are left to the other students for your grading and assignment completion, meaning that some of your assignments can hang there for weeks before ever getting reviewed."])

In [81]:
Text1="Professors need to undergo a presentation and Instruction Design skills. They were talking a lot and everything was going over my head, the reason was there were not visual prompt for the examples, specifically in the last week."
Text1=Text1.lower()
a=[]
a.append(Text1)
pred1,raw_op_1 = model.predict(a)

In [85]:
import aspect_based_sentiment_analysis as absa
nlp = absa.load()

Some layers from the model checkpoint at absa/classifier-rest-0.2 were not used when initializing BertABSClassifier: ['dropout_379']
- This IS expected if you are initializing BertABSClassifier from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertABSClassifier from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of BertABSClassifier were not initialized from the model checkpoint at absa/classifier-rest-0.2 and are newly initialized: ['dropout_151']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [86]:
Text2="I didn't like that course because of:-weak explanations of algorithms-almost all assignments I googled, I spent all my time googling these algorithms-when code is submitted and something goes wrong It is not possible to understand what is really happened"
Text2=Text2.lower()
a=[]
a.append(Text2)
pred2,raw_op_2 = model.predict(a)
print(pred2)

course,assignments = nlp(Text2, aspects=['course', 'assignments'])
print(absa.summary(course))
print(absa.summary(assignments))


[[{'i': 'O'}, {"didn't": 'O'}, {'like': 'O'}, {'that': 'O'}, {'course': 'B-Course'}, {'because': 'O'}, {'of:-weak': 'O'}, {'explanations': 'O'}, {'of': 'O'}, {'algorithms-almost': 'O'}, {'all': 'O'}, {'assignments': 'B-Content'}, {'i': 'O'}, {'googled,': 'O'}, {'i': 'O'}, {'spent': 'O'}, {'all': 'O'}, {'my': 'O'}, {'time': 'O'}, {'googling': 'O'}, {'these': 'O'}, {'algorithms-when': 'O'}, {'code': 'O'}, {'is': 'O'}, {'submitted': 'O'}, {'and': 'O'}, {'something': 'O'}, {'goes': 'O'}, {'wrong': 'O'}, {'it': 'O'}, {'is': 'O'}, {'not': 'O'}, {'possible': 'O'}, {'to': 'O'}, {'understand': 'O'}, {'what': 'O'}, {'is': 'O'}, {'really': 'O'}, {'happened': 'O'}]]
Sentiment.negative for "course"
Scores (neutral/negative/positive): [0.004 0.984 0.011]
None
Sentiment.negative for "assignments"
Scores (neutral/negative/positive): [0.053 0.938 0.009]
None


In [89]:
Text3="I was disappointed because the name is misleading. The course provides a good introduction & overview of the responsibilities of the CTO, but has very little specifically digital content. It deals with two-speed IT in a single short lecture, so of course the treatment is superficial. It is easy to find more in-depth material freely available, on the McKinsey website for example."
Text3=Text3.lower()
a=[]
a.append(Text3)
pred3,raw_op_2 = model.predict(a)
print(pred3)

course,material = nlp(Text3, aspects=['course', 'material'])
print(absa.summary(course))
print(absa.summary(material))


[[{'i': 'O'}, {'was': 'O'}, {'disappointed': 'O'}, {'because': 'O'}, {'the': 'O'}, {'name': 'O'}, {'is': 'O'}, {'misleading.': 'O'}, {'the': 'O'}, {'course': 'B-Course'}, {'provides': 'O'}, {'a': 'O'}, {'good': 'O'}, {'introduction': 'O'}, {'&': 'O'}, {'overview': 'O'}, {'of': 'O'}, {'the': 'O'}, {'responsibilities': 'O'}, {'of': 'O'}, {'the': 'O'}, {'cto,': 'O'}, {'but': 'O'}, {'has': 'O'}, {'very': 'O'}, {'little': 'O'}, {'specifically': 'O'}, {'digital': 'O'}, {'content.': 'O'}, {'it': 'O'}, {'deals': 'O'}, {'with': 'O'}, {'two-speed': 'O'}, {'it': 'O'}, {'in': 'O'}, {'a': 'O'}, {'single': 'O'}, {'short': 'O'}, {'lecture,': 'O'}, {'so': 'O'}, {'of': 'O'}, {'course': 'I-Course'}, {'the': 'O'}, {'treatment': 'O'}, {'is': 'O'}, {'superficial.': 'O'}, {'it': 'O'}, {'is': 'O'}, {'easy': 'O'}, {'to': 'O'}, {'find': 'O'}, {'more': 'O'}, {'in-depth': 'O'}, {'material': 'B-Content'}, {'freely': 'O'}, {'available,': 'O'}, {'on': 'O'}, {'the': 'O'}, {'mckinsey': 'O'}, {'website': 'O'}, {'for'

In [92]:
Text4="I love the professors, but unfortunately, I do not feel much connection to me (not really seeing so much importance for me to identifying if a company is making fake financial statements; I am not an investor, and my everyday work does not really have so much to do with financial statements).The Managerial Accounting (week 4) transforms my mindset!! I really appreciate that. That one single week of course can literally help so many companies/NGOs. That week 4 should be promoted more!!"
Text4=Text4.lower()
a=[]
a.append(Text4)
pred4,raw_op_4 = model.predict(a)
print(pred4)

professors,course,accounting = nlp(Text4, aspects=['professors', 'course','accounting'])
print(absa.summary(professors))
print(absa.summary(course))
print(absa.summary(accounting))


[[{'i': 'O'}, {'love': 'O'}, {'the': 'O'}, {'professors,': 'O'}, {'but': 'O'}, {'unfortunately,': 'O'}, {'i': 'O'}, {'do': 'O'}, {'not': 'O'}, {'feel': 'O'}, {'much': 'O'}, {'connection': 'O'}, {'to': 'O'}, {'me': 'O'}, {'(not': 'O'}, {'really': 'O'}, {'seeing': 'O'}, {'so': 'O'}, {'much': 'O'}, {'importance': 'O'}, {'for': 'O'}, {'me': 'O'}, {'to': 'O'}, {'identifying': 'O'}, {'if': 'O'}, {'a': 'O'}, {'company': 'O'}, {'is': 'O'}, {'making': 'O'}, {'fake': 'O'}, {'financial': 'O'}, {'statements;': 'O'}, {'i': 'O'}, {'am': 'O'}, {'not': 'O'}, {'an': 'O'}, {'investor,': 'O'}, {'and': 'O'}, {'my': 'O'}, {'everyday': 'O'}, {'work': 'O'}, {'does': 'O'}, {'not': 'O'}, {'really': 'O'}, {'have': 'O'}, {'so': 'O'}, {'much': 'O'}, {'to': 'O'}, {'do': 'O'}, {'with': 'O'}, {'financial': 'O'}, {'statements).the': 'O'}, {'managerial': 'O'}, {'accounting': 'O'}, {'(week': 'O'}, {'4)': 'O'}, {'transforms': 'O'}, {'my': 'O'}, {'mindset!!': 'O'}, {'i': 'O'}, {'really': 'O'}, {'appreciate': 'O'}, {'tha

In [98]:
Text5="First of all, I really enjoyed Professor Ittner presentations. Sadly, his presentation was only 25% of the course. Professor Ittner was very direct but also was very engaging in how he delivered his content."
#  I had a good feeling that I could really understand the material he was teaching. If I could give a rating for just Professor Ittner, I would give a 5 star rating. Now, this is why I am giving this course a 1 star rating. But I do have to say, going into the depths of Accounting Analytics is not easy. You really need to have a good grasp of Accounting otherwise you will get lost. Professor Bushee did not make the first 3 weeks really easy to get through. While I enjoyed his examples & case studies, he simply did not do a great job of really explaining and illustrating the concepts he was teaching. And worst of all, he setup a digitally generated classroom where he had his computer generated students ask pointless questions, make pointless statements, or answer with pointless statements/answers. I really wish I had the time to analyze how much of the course time he spent doing this, I would estimate that its around 10% but it could very well be higher. There are so many different examples to pick form, but I ll just pick a few here. I didn't read the Iliad or Odyssey some I'm not going to ask you to in reference why in regression there are Greek letter used for coefficients (its a fair assumption that someone taking this class understands coefficients), a 30 second exchange where a PC generate student says that they're "stocked" to run some regression followed by Proffessor Bushee informing us that "we're going to run some narly regressions, Professor Bushee saying he doesn t know how to perform Earning Management Analysis on Mars but he certainly knows how to do it on Earth, and a joke why people dont like accountants. These segments add absolutely zero value. They lengthen the time of the course. To make maters worse, I m sure Proffessor Bushee had to spend even more time creating this additional interaction together."
Text5=Text5.lower()
a=[]
a.append(Text5)
pred5,raw_op_5 = model.predict(a)
print(pred5)

# professors,course,accounting = nlp(Text4, aspects=['professors', 'course','accounting'])
# print(absa.summary(professors))
# print(absa.summary(course))
# print(absa.summary(accounting))


[[{'first': 'O'}, {'of': 'O'}, {'all,': 'O'}, {'i': 'O'}, {'really': 'O'}, {'enjoyed': 'O'}, {'professor': 'O'}, {'ittner': 'O'}, {'presentations.': 'O'}, {'sadly,': 'O'}, {'his': 'O'}, {'presentation': 'O'}, {'was': 'O'}, {'only': 'O'}, {'25%': 'O'}, {'of': 'O'}, {'the': 'O'}, {'course.': 'O'}, {'professor': 'O'}, {'ittner': 'O'}, {'was': 'O'}, {'very': 'O'}, {'direct': 'O'}, {'but': 'O'}, {'also': 'O'}, {'was': 'O'}, {'very': 'O'}, {'engaging': 'O'}, {'in': 'O'}, {'how': 'O'}, {'he': 'O'}, {'delivered': 'O'}, {'his': 'O'}, {'content.': 'O'}]]


In [99]:
pred5

[[{'first': 'O'},
  {'of': 'O'},
  {'all,': 'O'},
  {'i': 'O'},
  {'really': 'O'},
  {'enjoyed': 'O'},
  {'professor': 'O'},
  {'ittner': 'O'},
  {'presentations.': 'O'},
  {'sadly,': 'O'},
  {'his': 'O'},
  {'presentation': 'O'},
  {'was': 'O'},
  {'only': 'O'},
  {'25%': 'O'},
  {'of': 'O'},
  {'the': 'O'},
  {'course.': 'O'},
  {'professor': 'O'},
  {'ittner': 'O'},
  {'was': 'O'},
  {'very': 'O'},
  {'direct': 'O'},
  {'but': 'O'},
  {'also': 'O'},
  {'was': 'O'},
  {'very': 'O'},
  {'engaging': 'O'},
  {'in': 'O'},
  {'how': 'O'},
  {'he': 'O'},
  {'delivered': 'O'},
  {'his': 'O'},
  {'content.': 'O'}]]

In [58]:
import aspect_based_sentiment_analysis as absa

nlp = absa.load()
text = ("The course is fine as far as content. My complaint is that there is no one to contact regarding papers or questions. You are left to the other students for your grading and assignment completion, meaning that some of your assignments can hang there for weeks before ever getting reviewed.")

course,assignments = nlp(text, aspects=['course', 'assignments'])
print(absa.summary(course))
print(absa.summary(assignments))

Some layers from the model checkpoint at absa/classifier-rest-0.2 were not used when initializing BertABSClassifier: ['dropout_379']
- This IS expected if you are initializing BertABSClassifier from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertABSClassifier from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of BertABSClassifier were not initialized from the model checkpoint at absa/classifier-rest-0.2 and are newly initialized: ['dropout_113']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Sentiment.positive for "course"
Scores (neutral/negative/positive): [0.003 0.001 0.996]
None
Sentiment.neutral for "assignments"
Scores (neutral/negative/positive): [0.59  0.401 0.008]
None


In [55]:
absa.summary(course)

Sentiment.positive for "course"
Scores (neutral/negative/positive): [0.003 0.001 0.996]


In [56]:
absa.summary(assignments)

Sentiment.neutral for "assignments"
Scores (neutral/negative/positive): [0.59  0.401 0.008]


In [52]:
!pip install aspect-based-sentiment-analysis

     |████████████████████████████████| 266kB 9.2MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 92kB 7.7MB/s 
     |████████████████████████████████| 163kB 20.6MB/s 
     |████████████████████████████████| 81kB 8.0MB/s 
     |████████████████████████████████| 2.9MB 19.6MB/s 
     |████████████████████████████████| 81kB 9.8MB/s 
     |████████████████████████████████| 133kB 44.0MB/s 
     |████████████████████████████████| 51kB 6.4MB/s 
     |████████████████████████████████| 112kB 42.1MB/s 
  Created wheel for optuna: filename=optuna-2.3.0-cp36-none-any.whl size=359761 sha256=50c55399bf4f19d1e308023d2e5becaa2e8b38ecb5ce9223584adbcbae621750
  Stored in directory: /root/.cache/pip/wheels/fa/91/19/64b0ec6b964f89c0695a9dc6db6f851d0b54c5381a5c9cadfb
Successfully built optuna
  Created wheel for PrettyTable: filename=prettytable-0.7.2-cp36-none-any.whl size=13700 sha25